In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
from itertools import permutations
from pathlib import Path
from dolfin import *
import matplotlib.pyplot as plt
import numpy as np


from multirat.multicompartment import *

In [6]:
from multirat.meshprocessing import Domain
from multirat.parameters import multicompartment_parameters
from multirat.boundary import DirichletBoundary, RobinBoundary

def create_mesh(n, x0=-1., y0=-1., x1=1., y1=1.):
    return Domain(
        mesh=RectangleMesh(Point(x0, y0), Point(x1, y1), n, n),
        subdomains=None,
        boundaries=None,
    )

In [4]:
compartments = ['ecs', 'pvs_arteries', 'pvs_capillaries', 'pvs_veins']
params = multicompartment_parameters(compartments)

domain = create_mesh(40)
V = pressure_functionspace(domain.mesh, degree=1, compartments=compartments)

# Boundary conditions
L_bdry = params["hydraulic_conductivity_bdry"]
p_bdry = params["pressure_boundaries"]
boundaries = {
    "ecs": [RobinBoundary(L_bdry["ecs"], p_bdry["ecs"], "everywhere")],
    "pvs_arteries": [RobinBoundary(L_bdry["pvs_arteries"], p_bdry["pvs_arteries"], "everywhere")],
    "pvs_capillaries": [],
    "pvs_veins": [DirichletBoundary(p_bdry["pvs_veins"], "everywhere")],
}

results_path = Path("../results/pressure").resolve()
p = solve_pressure(domain, V, compartments, boundaries, params)
store_pressure(results_path, p, compartments, store_xdmf=True)


solute_bcs = {
    "ecs": [DirichletBoundary(Constant(0.0), "everywhere")],
    "pvs_arteries": [DirichletBoundary(Constant(0.0), "everywhere")],
    "pvs_capillaries": [DirichletBoundary(Constant(0.0), "everywhere")],
    "pvs_veins": [DirichletBoundary(Constant(0.0), "everywhere")],
}

dt  = 600  # 10min timestep.
T = 1 * 3600 # 4h
N = int(T / dt) + 1

c_init = Expression("exp(-(pow(x[0] - c[0], 2) + pow(x[1]-c[1], 2)) / (length * length))",
                    length=Constant(0.4), c=Constant((0.5, 0.0)), degree=2)

c0 = Function(V)
assign(c0.sub(0), project(c_init, V.sub(0).collapse()))


times = np.array([dt * i for i in range(N)])
C, mass = solve_solute(c0, p, dt, T, domain, V, compartments, solute_bcs, params)

plt.figure()
plt.plot(times, mass, "o-")
plt.show()

NameError: name 'pressure_functionspace' is not defined